# 脚本编程与系统管理
---

### 通过重定向/管道/文件接受输入

使用 `fileinput` 模块可以接受从命令行输出、重定向文件，或命令行传递的文件名列表等输入。

```python
import fileinput

with fileinput.input() as f_input:
    for line in list(f_input)[:20]:
        print(line, end='')
```

将以上代码保存后在命令行如下执行  
`$ ls | ./filein.py`           # 打印目录列表到屏幕输出  
`$ ./filein.py /etc/passwd`    # 读取 /etc/passwd 到屏幕输出  
`$ ./filein.py < /etc/passwd`  # 读取 /etc/passwd 到屏幕输出

---

### 终止程序并给出错误信息

In [1]:
import sys
raise SystemExit('It failed!')

SystemExit: It failed!

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


---

### 解析命令行选项

可以使用标准库的 `argparse` 模块，首先要创建一个 `ArgumentParser` 实例， 并使用 `add_argument` 方法声明你想要支持的选项。 在每个 `add_argument` 调用中，`dest` 参数指定解析结果被指派给属性的名字。 `metavar` 参数被用来生成帮助信息。`action` 参数指定跟属性对应的处理逻辑， 通常的值为 `store` ，被用来存储某个值或将多个参数值收集到一个列表中。 

```python
import argparse

parser = argparse.ArgumentParser(description='Search some files')

parser.add_argument(dest='filenames',metavar='filename', nargs='*')

parser.add_argument('-p', '--pat', metavar='pattern', 
                    required=True, dest='patterns', 
                    action='append',
                    help='text pattern to search for')

parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)
```

将以上代码保存后在命令行如下执行  
```
$ python3 search.py -h
usage: search.py [-h] [-p pattern] [-v] [-o OUTFILE] [--speed {slow,fast}]
                 [filename [filename ...]]

Search some files

positional arguments:
  filename

optional arguments:
  -h, --help            show this help message and exit
  -p pattern, --pat pattern
                        text pattern to search for
  -v                    verbose mode
  -o OUTFILE            output file
  --speed {slow,fast}   search speed

$ python3 search.py foo.txt bar.txt
usage: search.py [-h] -p pattern [-v] [-o OUTFILE] [--speed {fast,slow}]
                 [filename [filename ...]]
search.py: error: the following arguments are required: -p/--pat

$ python3 search.py -v -p spam --pat=eggs foo.txt bar.txt
filenames = ['foo.txt', 'bar.txt']
patterns  = ['spam', 'eggs']
verbose   = True
outfile   = None
speed     = slow

$ python3 search.py -v -p spam --pat=eggs foo.txt bar.txt -o results
filenames = ['foo.txt', 'bar.txt']
patterns  = ['spam', 'eggs']
verbose   = True
outfile   = results
speed     = slow

$ python3 search.py -v -p spam --pat=eggs foo.txt bar.txt -o results --speed=fast
filenames = ['foo.txt', 'bar.txt']
patterns  = ['spam', 'eggs']
verbose   = True
outfile   = results
speed     = fast

```

---

### 运行时弹出密码输入提示

In [2]:
import getpass

# 该函数依次检测环境变量 LOGNAME、USER、LNAME 和 USERNAME，返回其中第一个非空的值
user = getpass.getuser()
# 该函数键入
passwd = getpass.getpass('Enter your password: ')

def svc_login(ur, pw):
    if pw == 'admin':
        return True
    else:
        return False

if svc_login(user, passwd):
   print('Right!')
else:
   print('Wrong!')

Enter your password: ········
Right!


---

### 获取终端的大小

In [3]:
import os
sz = os.get_terminal_size()

In [4]:
sz

os.terminal_size(columns=80, lines=24)

In [5]:
sz.columns

80

In [6]:
sz.lines

24

---

### 执行外部命令并获取它的输出

In [7]:
import subprocess
out_bytes = subprocess.check_output(['ls'])

In [8]:
print(out_bytes.decode('utf-8'))

函数.ipynb
元编程.ipynb
文件与IO.ipynb
网络与Web编程.ipynb
并发编程.ipynb
模块与包.ipynb
类与对象.ipynb
字符串和文本.ipynb
数字日期和时间.ipynb
数据结构和算法.ipynb
迭代器与生成器.ipynb
脚本编程与系统管理.ipynb



---

### 复制或者移动文件和目录

```python
import shutil

# 拷贝文件 src 到 dst，包括文件和权限
shutil.copy(src, dst)

# 在 copy 方法基础上，额外拷贝修改时间和最后访问时间
shutil.copy2(src, dst)

# 拷贝目录
shutil.copytree(src, dst)

# 移动文件
shutil.move(src, dst)
```

---

### 创建和解压归档文件

```python
import shutil

# 解压
shutil.unpack_archive('Python-3.3.0.tgz')

# 将 Python-3.3.0 目录压缩成 py33.zip
shutil.make_archive('py33','zip','Python-3.3.0')
```

---

### 通过文件名查找文件

In [9]:
import os

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            full_path = os.path.join(start, relpath, name)
            print(os.path.normpath(os.path.abspath(full_path)))

In [10]:
findfile('./', '脚本编程与系统管理.ipynb')

/Users/zkqiang/Documents/GitHub/Python-Cookbook-Note/脚本编程与系统管理.ipynb


---

### 读取配置文件

```
; config.ini
; Sample configuration file

[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local

# Setting related to debug configuration
[debug]
log_errors=true
show_warnings=False

[server]
port: 8080
nworkers: 32
pid-file=/tmp/spam.pid
root=/www/root
signature:
    =================================
    Brought to you by the Python Cookbook
    =================================
```

```python
>>> from configparser import ConfigParser
>>> cfg = ConfigParser()
>>> cfg.read('config.ini')
['config.ini']
>>> cfg.sections()
['installation', 'debug', 'server']
>>> cfg.get('installation','library')
'/usr/local/lib'
>>> cfg.getboolean('debug','log_errors')

True
>>> cfg.getint('server','port')
8080
>>> cfg.getint('server','nworkers')
32
>>> print(cfg.get('server','signature'))

\=================================
Brought to you by the Python Cookbook
\=================================
```